### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
# Dependencies and Setup
import pandas as pd

# Import data from csv files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [44]:
#rename columns for cleaner syntex
school_data_complete.columns=['ID', 'Name', 'Gender', 'Grade', 'School', 
                              'Reading', 'Math', 'School ID', 'Type', 'Size', 'Budget']

#group by school types
district_group = school_data_complete.loc[school_data_complete['Type'] == 'District']
charter_analyzed = school_data_complete.loc[school_data_complete['Type'] == 'Charter']



#define funtion to analyze dataframes 
def school_type(dataframe):

    #Store school Type into variable
    school_type = dataframe.Type.unique()
    
    #calculate number of schools and students
    number_schools = dataframe['School'].nunique()
    number_students = dataframe['Name'].count()

    #calculate total budget
    school_group = dataframe.groupby(['School'])
    total_budget = school_group['Budget'].max().sum()

    #calculate average math and reading scores
    math_average = dataframe['Math'].mean()
    reading_average = dataframe['Reading'].mean()

    #calculate overall passing rate
    passing_rate = (math_average + reading_average)/2

    #calculate percentage of students with a passing math score
    math_pass = dataframe.loc[school_data_complete['Math'] > 69, :]
    math_pass_perc = round((math_pass['Math'].count() / number_students), 4) * 100

    #calculate percentage of students with a passing reading score
    read_score = (dataframe.loc[school_data_complete['Reading'] > 69, :])
    read_score_perc = round((read_score['Reading'].count() / number_students), 4) * 100

    #create dataframe for all above values
    district_analyzed = pd.DataFrame({
                        'Type':[school_type[0]],
                        'Schools':[number_schools],
                        'Students':[number_students],
                        'Total Budget':[total_budget],
                        'Average Math Score':[round(math_average)],
                        'Average Reading Score':[round(reading_average)],
                        'Overall Pass Rate':[passing_rate],
                        'Math Passed Perc.':[math_pass_perc],
                        'Reading Passed Perc.':[read_score_perc]
    })

    #swap rows with columns
    #district_analyzed = district_analyzed.transpose()
    return district_analyzed 

#call funtion for charter and district schools and store in seperate databases
district_analyzed = school_type(district_group)
charter_analyzed = school_type(charter_analyzed)

#merge district & charter database & add column formating
type_summary =district_analyzed.merge(charter_analyzed, how='outer')
type_summary['Students'] = type_summary['Students'].map("{:,}".format)
type_summary['Total Budget'] = type_summary['Total Budget'].map("${:,}".format)
type_summary['Math Passed Perc.'] = type_summary['Math Passed Perc.'].map("{:.2f}%".format)
type_summary['Reading Passed Perc.'] = type_summary['Reading Passed Perc.'].map("{:.2f}%".format)
type_summary['Overall Pass Rate'] = type_summary['Overall Pass Rate'].map("{:.2f}%".format)
type_summary

,Type,Schools,Students,Total Budget,Average Math Score,Average Reading Score,Overall Pass Rate,Math Passed Perc.,Reading Passed Perc.
0,District,7,"26,976","$17,347,923",77,81,78.97%,66.52%,80.91%
1,Charter,8,"12,194","$7,301,505",83,84,83.65%,93.70%,96.65%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [45]:
school_new_index = school_data_complete.set_index(["School", "Type"])
school_summary = school_data_complete['School'].unique()
#convert array(school_summary) into a list so it can be iterated over
school_summary = school_summary.tolist()

#create a funtion to make Dataframes for all schools, then each grade

def school_stats(school_summary, dataframe):
    budget_list = []
    Budget = []
    school_stats_list = []

    #loop through list of schools
    for school in range(len(school_summary)): 

        #set a variable for each school stat
        school_name = (school_summary[school])
        school_type_raw = school_new_index.loc[school_name].index
        school_type = school_type_raw[1]
        school_budget = (dataframe.loc[(school_name), 'Budget'].max())
        school_size = (dataframe.loc[(school_name), 'Size'].max())
        math_av = round((dataframe.loc[(school_name), 'Math'].mean()), 0)
        reading_av = round((dataframe.loc[(school_name), 'Reading'].mean()), 0)
        math_pass = dataframe.loc[(school_name), dataframe['Math'] > 69, :]
        math_pass_perc = round(((math_pass['Math'].count() / school_size)*100), 2)
        read_pass = dataframe.loc[(school_name), dataframe['Reading'] > 69, :]
        read_pass_perc = round(((read_pass['Reading'].count() / school_size)*100), 2)
        overall_pass = round(((math_pass_perc + read_pass_perc) / 2), 2) 

        #calculate all desired variables and store them in a new list
        per_student_budget = (school_budget/school_size)

        #create a list for current school and all it's stats
        Budget = [school_type, school_size, school_budget,  
        per_student_budget, math_av, reading_av, 
        math_pass_perc, read_pass_perc, overall_pass]

        #append current list of school stats to list containing all school stats
        budget_list.append(Budget)

    #create a list for columns
    rows = ['School Type','Total Students', 'Total School Budget',
        'Per Student Budget', 'Average Math Score', 'Average Reading Score',
        '%Passing Math', '%Passing Reading', '%Overall Passing Rate']

    #create a dictionary then a dataframe 
    school_zip = zip(school_summary, budget_list)
    school_dict = dict(school_zip)
    school_dict
    school_summary_df = pd.DataFrame(school_dict, index=rows)
    school_summary_df = school_summary_df.transpose()
  
    return school_summary_df 

#Call function to store summary of each school in new database
school_analyzed_df = school_stats(school_summary, school_new_index)
school_stats_df = school_stats(school_summary, school_new_index)

#format database columns
school_stats_df['Total School Budget'] = school_stats_df['Total School Budget'].map("${:,.0f}".format)
school_stats_df['Per Student Budget'] = school_stats_df['Per Student Budget'].map("${:,.0f}".format)
school_stats_df['%Passing Math'] = school_stats_df['%Passing Math'].map("{:,.2f}%".format)
school_stats_df['%Passing Reading'] = school_stats_df['%Passing Reading'].map("{:,.2f}%".format)
school_stats_df['%Overall Passing Rate'] = school_stats_df['%Overall Passing Rate'].map("{:,.2f}%".format)

school_stats_df.sort_values('School Type')


C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:22: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing Rate
Shelton High School,Charter,1761,"$1,056,600",$600,83,84,93.87%,95.85%,94.86%
Griffin High School,Charter,1468,"$917,500",$625,83,84,93.39%,97.14%,95.26%
Wilson High School,Charter,2283,"$1,319,574",$578,83,84,93.87%,96.54%,95.21%
Cabrera High School,Charter,1858,"$1,081,356",$582,83,84,94.13%,97.04%,95.58%
Holden High School,Charter,427,"$248,087",$581,84,84,92.51%,96.25%,94.38%
Pena High School,Charter,962,"$585,858",$609,84,84,94.59%,95.95%,95.27%
Wright High School,Charter,1800,"$1,049,400",$583,84,84,93.33%,96.61%,94.97%
Thomas High School,Charter,1635,"$1,043,130",$638,83,84,93.27%,97.31%,95.29%
Huang High School,District,2917,"$1,910,635",$655,77,81,65.68%,81.32%,73.50%
Figueroa High School,District,2949,"$1,884,411",$639,77,81,65.99%,80.74%,73.36%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [46]:
#Sort and display the top five schools in overall passing rate
school_stats_df['%Overall Passing Rate'].sort_values(ascending=False).head(5)

Cabrera High School    95.58%
Thomas High School     95.29%
Pena High School       95.27%
Griffin High School    95.26%
Wilson High School     95.21%
Name: %Overall Passing Rate, dtype: object

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [47]:
#Sort and display the five worst-performing schools
school_stats_df['%Overall Passing Rate'].sort_values().head(5)

Rodriguez High School    73.30%
Figueroa High School     73.36%
Huang High School        73.50%
Johnson High School      73.64%
Hernandez High School    73.81%
Name: %Overall Passing Rate, dtype: object

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [48]:
#create a list of the grades to loop through
grade_list = school_data_complete['Grade'].unique()
grade_list = grade_list.tolist()

#loop through grades 
for i in grade_list:
    
    #create a new dataframe based off of current grade in loop
    grade_df = school_new_index.loc[school_new_index['Grade'] == i, :]
    
    #send school list and grade dataframe to analyzing function
    grade_data = school_stats(school_summary, grade_df)  
    
    #make databases for each grade containing av read and math scores
    if i == '9th':
        grade9_reading = pd.DataFrame(grade_data['Average Reading Score']) 
        grade9_math = pd.DataFrame(grade_data['Average Math Score'])
        grade9_reading = grade9_reading.rename(columns={'Average Reading Score':'Average Reading Score 9th'})
        grade9_math = grade9_math.rename(columns={'Average Math Score':'Average Math Score 9th'})
        
    elif i == '10th':
        grade10_reading = pd.DataFrame(grade_data['Average Reading Score'])
        grade10_math = pd.DataFrame(grade_data['Average Math Score']) 
        grade10_reading = grade10_reading.rename(columns={'Average Reading Score':'Average Reading Score 10th'})
        grade10_math = grade10_math.rename(columns={'Average Math Score':'Average Math Score 10th'})
        

    elif i == '11th':
        grade11_reading = pd.DataFrame(grade_data['Average Reading Score'])
        grade11_math = pd.DataFrame(grade_data['Average Math Score'])
        grade11_reading = grade11_reading.rename(columns={'Average Reading Score':'Average Reading Score 11th'})
        grade11_math = grade11_math.rename(columns={'Average Math Score':'Average Math Score 11th'})
        
    elif i == '12th':
        grade12_reading = pd.DataFrame(grade_data['Average Reading Score'])
        grade12_math = pd.DataFrame(grade_data['Average Math Score']) 
        grade12_reading = grade12_reading.rename(columns={'Average Reading Score':'Average Reading Score 12th'})
        grade12_math = grade12_math.rename(columns={'Average Math Score':'Average Math Score 12th'})
       
#merge math databases into 1 database
grade_math = grade9_math.merge(grade10_math, how='outer', left_index=True, right_index=True)
grade_math = grade_math.merge(grade11_math, how='outer', left_index=True, right_index=True)
grade_math = grade_math.merge(grade12_math, how='outer', left_index=True, right_index=True)
grade_math    


C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:22: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Deepak\Anaconda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:23: PerformanceWarning: indexing past lexsort depth may impact performance.


,Average Math Score 9th,Average Math Score 10th,Average Math Score 11th,Average Math Score 12th
Huang High School,77,76,76,77
Figueroa High School,76,77,77,77
Shelton High School,83,83,83,84
Hernandez High School,77,77,77,77
Griffin High School,82,84,84,83
Wilson High School,83,84,83,83
Cabrera High School,83,83,83,83
Bailey High School,77,77,78,76
Holden High School,84,83,85,83
Pena High School,84,83,84,84


## Reading Score by Grade 

In [49]:
#merge reading databases into 1 database
grade_reading = grade9_reading.merge(grade10_reading, how='outer', left_index=True, right_index=True)
grade_reading = grade_reading.merge(grade11_reading, how='outer', left_index=True, right_index=True)
grade_reading = grade_reading.merge(grade12_reading, how='outer', left_index=True, right_index=True)
grade_reading

,Average Reading Score 9th,Average Reading Score 10th,Average Reading Score 11th,Average Reading Score 12th
Huang High School,81,82,81,80
Figueroa High School,81,81,81,81
Shelton High School,84,83,84,83
Hernandez High School,81,81,81,81
Griffin High School,83,84,84,84
Wilson High School,84,84,84,84
Cabrera High School,84,84,84,84
Bailey High School,81,81,81,81
Holden High School,84,83,84,85
Pena High School,84,84,84,85


* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [50]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#create new dataframe to modify
school_spending_df = school_analyzed_df

#del unwanted columns
del school_spending_df['Total Students']
del school_spending_df['Total School Budget']

#create new dataframe column that contains bins
school_spending_df['Per Student Spending'] = pd.cut(school_analyzed_df['Per Student Budget'], spending_bins, labels = group_names)

#sort Per Student values and add formating
school_spending = school_spending_df.sort_values(['Per Student Spending'])
school_spending['Per Student Budget'] = school_spending['Per Student Budget'].map("${:,}".format)
school_spending

,School Type,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing Rate,Per Student Spending
Wilson High School,Charter,$578.0,83,84,93.87,96.54,95.21,<$585
Cabrera High School,Charter,$582.0,83,84,94.13,97.04,95.58,<$585
Holden High School,Charter,$581.0,84,84,92.51,96.25,94.38,<$585
Wright High School,Charter,$583.0,84,84,93.33,96.61,94.97,<$585
Shelton High School,Charter,$600.0,83,84,93.87,95.85,94.86,$585-615
Pena High School,Charter,$609.0,84,84,94.59,95.95,95.27,$585-615
Figueroa High School,District,$639.0,77,81,65.99,80.74,73.36,$615-645
Griffin High School,Charter,$625.0,83,84,93.39,97.14,95.26,$615-645
Bailey High School,District,$628.0,77,81,66.68,81.93,74.31,$615-645
Rodriguez High School,District,$637.0,77,81,66.37,80.22,73.3,$615-645


## Scores by School Size

* Perform the same operations as above, based on school size.

In [51]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size_df = school_stats_df 

del school_size_df['Total School Budget']
school_size_df['School Size'] = pd.cut(school_size_df['Total Students'], size_bins, labels = group_names)
school_size_df.sort_values(['School Size'])

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing Rate,School Size
Holden High School,Charter,427,"$248,087",$581,84,84,92.51%,96.25%,94.38%,Small (<1000)
Pena High School,Charter,962,"$585,858",$609,84,84,94.59%,95.95%,95.27%,Small (<1000)
Shelton High School,Charter,1761,"$1,056,600",$600,83,84,93.87%,95.85%,94.86%,Medium (1000-2000)
Griffin High School,Charter,1468,"$917,500",$625,83,84,93.39%,97.14%,95.26%,Medium (1000-2000)
Cabrera High School,Charter,1858,"$1,081,356",$582,83,84,94.13%,97.04%,95.58%,Medium (1000-2000)
Wright High School,Charter,1800,"$1,049,400",$583,84,84,93.33%,96.61%,94.97%,Medium (1000-2000)
Thomas High School,Charter,1635,"$1,043,130",$638,83,84,93.27%,97.31%,95.29%,Medium (1000-2000)
Huang High School,District,2917,"$1,910,635",$655,77,81,65.68%,81.32%,73.50%,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411",$639,77,81,65.99%,80.74%,73.36%,Large (2000-5000)
Hernandez High School,District,4635,"$3,022,020",$652,77,81,66.75%,80.86%,73.81%,Large (2000-5000)


## Scores by School Type

* Perform the same operations as above, based on school type.

In [52]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_spending_df = type_summary
school_spending_df['Per Student Spending'] = pd.cut(school_analyzed_df['Per Student Budget'], 
                                                spending_bins, labels = group_names)
school_spending_df.sort_values(['Per Student Spending'])
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size_df = school_stats_df 
school_size_df['School Size'] = pd.cut(school_size_df['Total Students'], size_bins, labels = group_names)
school_size_df.sort_values(['School Size'])


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing Rate,School Size
Holden High School,Charter,427,"$248,087",$581,84,84,92.51%,96.25%,94.38%,Small (<1000)
Pena High School,Charter,962,"$585,858",$609,84,84,94.59%,95.95%,95.27%,Small (<1000)
Shelton High School,Charter,1761,"$1,056,600",$600,83,84,93.87%,95.85%,94.86%,Medium (1000-2000)
Griffin High School,Charter,1468,"$917,500",$625,83,84,93.39%,97.14%,95.26%,Medium (1000-2000)
Cabrera High School,Charter,1858,"$1,081,356",$582,83,84,94.13%,97.04%,95.58%,Medium (1000-2000)
Wright High School,Charter,1800,"$1,049,400",$583,84,84,93.33%,96.61%,94.97%,Medium (1000-2000)
Thomas High School,Charter,1635,"$1,043,130",$638,83,84,93.27%,97.31%,95.29%,Medium (1000-2000)
Huang High School,District,2917,"$1,910,635",$655,77,81,65.68%,81.32%,73.50%,Large (2000-5000)
Figueroa High School,District,2949,"$1,884,411",$639,77,81,65.99%,80.74%,73.36%,Large (2000-5000)
Hernandez High School,District,4635,"$3,022,020",$652,77,81,66.75%,80.86%,73.81%,Large (2000-5000)
